In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect('/content/drive/My Drive/eng_subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('zipfiles',)]


In [ ]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

num
name
content


In [ ]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


In [ ]:
b_data = df.iloc[0, 2]

# here 2 represent the index of content column
# 0 represents the row number

In [ ]:
print(b_data)

b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x99V\x9fx\x96\xf0\x8c\x9e\x00\x00\x86\x9b\x01\x00;\x00\x00\x00The.Message.1976.REMASTERED.1080p.BluRay.x264-PiGNUS.EN.srt\xad\xbdm\x93\xdc\xc6\x91.\xfa\x9d\x11\xfc\x0f-}\xe1=\x11-\x9d\x06P\x85\x17\x9d\x8d\xd5%%[\xa4-Y>&u\x15>\xdf\xd0\xd3\x98\x19x\xfae\x0cts<\xfe\xf57\x9f\'\xb3\n\xd9\xa4\xbc\xbb\xf7\xc6Fl\xacELW\xa2\xaa\x90\x95\x95\xafO\x16/_l6\xdf\xe0\xff\xea\xf5f\xb3Y}\xf5\xd5\xbf\xaf\xf4AQ\xae7Mx\xf9\xe2\xd7\xfe|s\xbf\xea\x8f\xcf\xab\x8f\xe3n8\xadN\xc7\xfdx\x1cVO\xe3\xf9~\xf5\xf3\xe3p\xfc\xea\xfd/o>\xbc\xfb\xf0\xe3\xef\xde\xbf|\xf1\xfbi\x18Vo\xa6\xd3\xd3<L\xab\xe1\x1f\xe7\xe18\x8f\xa7\xe37\xab\xd3\xbc\xdb~-\xc3\x1e\xfe\xa7<|\xf9\xe2\xe5\x8bR_[~S\xd6\xeb\xa2k\xf3k\xe5A\xb7\xeeb\xf5\xf2\xc5\xbb\xe3\xea|?\xac\x8e\xfdaX\x9dnW?\x9cvk>8\x9c\xe6\xf3\xean\xeao\xc6\xd3ev\x8f~\x1a\xa6\x9b\xf1\xf6\xb2\xff\x1a\xe4\xabD\xbe*d\x11\xa5#_U\xeb\xaa\xd9`\xa6\xa7\xc3\xea\xa7\xcb}\x7f8\xf4F\xf9\xa7a\x9e\x87\xe3\x9d\xcc\\\xdf\x07B!\x13\xaa\xd61n<!\xd9\xaf\xd0\

In [ ]:
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    count += 1
    # Decompress the binary data using the zipfile module
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            file_name = zip_file.namelist()[0]
            with zip_file.open(file_name) as file_in_zip:
                file_size = file_in_zip.seek(0, 2)  # Get the size of the file
                file_in_zip.seek(0)  # Reset file pointer to the beginning
                # Calculate the ending position to read 20% of the data
                end_position = int(file_size * 0.2)  # 20% of the file size
                # Read only 20% of the data
                subtitle_content = file_in_zip.read(end_position)

    # Now 'subtitle_content' should contain the extracted subtitle content (first 20%)
    return subtitle_content.decode('latin-1')



In [ ]:
df['file_content'] = df['content'].apply(decode_method)

In [ ]:
df.head()

,num,name,content,file_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [ ]:
df.shape

(82498, 4)

In [ ]:
df['file_content'].iloc[0]

"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch any video online with Open-SUBTITLES\r\nFree Browser extension: osdb.link/ext\r\n\r\n2\r\n00:02:26,198 --> 00:02:29,953\r\nIn the name of God, the most gracious, the most Merciful.\r\n\r\n3\r\n00:02:31,072 --> 00:02:33,370\r\nFrom Muhammad, the Messenger of God\r\n\r\n4\r\n00:02:33,550 --> 00:02:36,047\r\nto Heraclius, the emperor of Byzantium.\r\n\r\n5\r\n00:02:36,407 --> 00:02:39,464\r\ngreetings to him who is the\r\nfollower of righteous guidance.\r\n\r\n6\r\n00:02:39,783 --> 00:02:42,591\r\nI bid you to hear the divine call.\r\n\r\n7\r\n00:02:43,160 --> 00:02:45,817\r\nI am the messenger of God to the people;\r\n\r\n8\r\n00:02:46,337 --> 00:02:48,784\r\naccept Islam for your salvation.\r\n\r\n9\r\n00:02:52,231 --> 00:02:54,709\r\nHe speaks of a new prophet in Arabia.\r\n\r\n10\r\n00:02:55,068 --> 00:02:57,825\r\nWas it like this when John, the Baptist\r\ncame to king Herod\r\n\r\n11\r\n00:02:58,145 --> 00:03:01,272\r\nout of the desert, 

In [ ]:
df1 = df.loc[:20000]

In [ ]:
import re
from bs4 import BeautifulSoup

In [ ]:
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Lowercase the text
    text = text.lower()
    # Convert accented vowels to their base form
    text = re.sub('[àáâãäå]', 'a', text)
    text = re.sub('[èéêë]', 'e', text)
    text = re.sub('[ìíîï]', 'i', text)
    text = re.sub('[òóôõö]', 'o', text)
    text = re.sub('[ùúûü]', 'u', text)


    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.strip()  # Remove leading/trailing whitespaces


In [ ]:
# Apply cleaning function to the 'text_column'
df1['cleaned_text'] = df1['file_content'].apply(clean_text)

<ipython-input-64-893d2fc5b37d>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
<ipython-input-65-805f2bedd43d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['cleaned_text'] = df1['file_content'].apply(clean_text)


In [ ]:
# Remove rows where 'cleaned_text' is empty
df1 = df1[df1['cleaned_text'] != '']

In [ ]:
df1['cleaned_text'].iloc[0]

'watch any video online with opensubtitles\r\nfree browser extension osdblinkext\r\n\r\n\r\n  \r\nin the name of god the most gracious the most merciful\r\n\r\n\r\n  \r\nfrom muhammad the messenger of god\r\n\r\n\r\n  \r\nto heraclius the emperor of byzantium\r\n\r\n\r\n  \r\ngreetings to him who is the\r\nfollower of righteous guidance\r\n\r\n\r\n  \r\ni bid you to hear the divine call\r\n\r\n\r\n  \r\ni am the messenger of god to the people\r\n\r\n\r\n  \r\naccept islam for your salvation\r\n\r\n\r\n  \r\nhe speaks of a new prophet in arabia\r\n\r\n\r\n  \r\nwas it like this when john the baptist\r\ncame to king herod\r\n\r\n\r\n  \r\nout of the desert crying about salvation\r\n\r\n\r\n  \r\nto muqawqis patriarch of alexandria\r\n\r\n\r\n  \r\nkisra emperor of persia\r\n\r\n\r\n  \r\nmuhammad calls you with the call of god\r\n\r\n\r\n  \r\naccept islam for your salvation\r\n\r\n\r\n  \r\nembrace islam\r\n\r\n\r\n  \r\nyou come out of the desert\r\nsmelling of camel and goat\r\n\r\n\r

In [ ]:
print(df1['cleaned_text'].iloc[0])

watch any video online with opensubtitles
free browser extension osdblinkext


  
in the name of god the most gracious the most merciful


  
from muhammad the messenger of god


  
to heraclius the emperor of byzantium


  
greetings to him who is the
follower of righteous guidance


  
i bid you to hear the divine call


  
i am the messenger of god to the people


  
accept islam for your salvation


  
he speaks of a new prophet in arabia


  
was it like this when john the baptist
came to king herod


  
out of the desert crying about salvation


  
to muqawqis patriarch of alexandria


  
kisra emperor of persia


  
muhammad calls you with the call of god


  
accept islam for your salvation


  
embrace islam


  
you come out of the desert
smelling of camel and goat


  
to tell persia where he should kneel


  
muhammad messenger of god


  
who gave him this authority


  
god sent muhammad
as a mercy to mankind


  
the scholars and historians of islam 
the university of al

In [ ]:
def remove(text):
    text = re.sub(r'\s+',' ',text)
    text = re.sub(r'\n+',' ',text).strip()
    return text

In [ ]:
df1.shape

(20001, 5)

In [ ]:
df1['cleaned_text'] = df1['cleaned_text'].apply(lambda x: remove(x))

In [ ]:
df1['cleaned_text'].iloc[0]

'watch any video online with opensubtitles free browser extension osdblinkext in the name of god the most gracious the most merciful from muhammad the messenger of god to heraclius the emperor of byzantium greetings to him who is the follower of righteous guidance i bid you to hear the divine call i am the messenger of god to the people accept islam for your salvation he speaks of a new prophet in arabia was it like this when john the baptist came to king herod out of the desert crying about salvation to muqawqis patriarch of alexandria kisra emperor of persia muhammad calls you with the call of god accept islam for your salvation embrace islam you come out of the desert smelling of camel and goat to tell persia where he should kneel muhammad messenger of god who gave him this authority god sent muhammad as a mercy to mankind the scholars and historians of islam the university of alazhar in cairo the high islamic congress of the shiat in lebanon the makers of this film honour the islam

In [ ]:
def chunk(document,token_window=500, overlap=100):
    tokens = document.split()

    chunks = []

    start_idx = 0

    while start_idx <len(tokens):
        end_idx = min(start_idx + token_window, len(tokens))
        chunks.append(' '.join(tokens[start_idx:end_idx]))
        start_idx+=token_window - overlap
    return chunks

In [ ]:
df1['chunks'] = df1['cleaned_text'].apply(lambda x: chunk(x))

In [ ]:
df1.head()

,num,name,content,file_content,cleaned_text,chunks
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther...",ah theres princess dawn and terry with the blo...,[ah theres princess dawn and terry with the bl...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'...",yumis cells episode extremely polite yumi yumi...,[yumis cells episode extremely polite yumi yum...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...",i watch any video online with opensubtitles fr...,[i watch any video online with opensubtitles f...


In [ ]:
df1["chunks"].iloc[0]

['watch any video online with opensubtitles free browser extension osdblinkext in the name of god the most gracious the most merciful from muhammad the messenger of god to heraclius the emperor of byzantium greetings to him who is the follower of righteous guidance i bid you to hear the divine call i am the messenger of god to the people accept islam for your salvation he speaks of a new prophet in arabia was it like this when john the baptist came to king herod out of the desert crying about salvation to muqawqis patriarch of alexandria kisra emperor of persia muhammad calls you with the call of god accept islam for your salvation embrace islam you come out of the desert smelling of camel and goat to tell persia where he should kneel muhammad messenger of god who gave him this authority god sent muhammad as a mercy to mankind the scholars and historians of islam the university of alazhar in cairo the high islamic congress of the shiat in lebanon the makers of this film honour the isla

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from sentence_transformers import SentenceTransformer, util
model =SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
def encode_text(text):
  embedding = model.encode(text)
  return embedding

In [ ]:
df2 = df1.loc[:1000,:]
df2.head()

,num,name,content,file_content,cleaned_text,chunks
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther...",ah theres princess dawn and terry with the blo...,[ah theres princess dawn and terry with the bl...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'...",yumis cells episode extremely polite yumi yumi...,[yumis cells episode extremely polite yumi yum...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...",i watch any video online with opensubtitles fr...,[i watch any video online with opensubtitles f...


In [ ]:
df2['embedding'] = df2['chunks'].apply(encode_text)
df2.head()

<ipython-input-80-c61f8fe49383>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['embedding'] = df2['chunks'].apply(encode_text)


,num,name,content,file_content,cleaned_text,chunks,embedding
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...,"[[-0.041853778, 0.12192455, -0.061531965, -0.0..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther...",ah theres princess dawn and terry with the blo...,[ah theres princess dawn and terry with the bl...,"[[-0.07573727, 0.012181397, 0.022164255, -0.07..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'...",yumis cells episode extremely polite yumi yumi...,[yumis cells episode extremely polite yumi yum...,"[[-0.13184361, -0.13974085, 0.045139585, -0.04..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...,"[[-0.08056183, -0.0905083, 0.058143258, -0.030..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...",i watch any video online with opensubtitles fr...,[i watch any video online with opensubtitles f...,"[[-0.07956292, -0.014649572, 0.011612225, -0.0..."


In [ ]:
df2.drop(columns=['content','file_content'],inplace=True)

<ipython-input-81-0e36bf84b4d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(columns=['content','file_content'],inplace=True)


In [ ]:
df1.drop(columns=['content','file_content'],inplace=True)

In [ ]:
df2.head()

,num,name,cleaned_text,chunks,embedding
0,9180533,the.message.(1976).eng.1cd,watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...,"[[-0.041853778, 0.12192455, -0.061531965, -0.0..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,ah theres princess dawn and terry with the blo...,[ah theres princess dawn and terry with the bl...,"[[-0.07573727, 0.012181397, 0.022164255, -0.07..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,yumis cells episode extremely polite yumi yumi...,[yumis cells episode extremely polite yumi yum...,"[[-0.13184361, -0.13974085, 0.045139585, -0.04..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...,"[[-0.08056183, -0.0905083, 0.058143258, -0.030..."
4,9180600,broker.(2022).eng.1cd,i watch any video online with opensubtitles fr...,[i watch any video online with opensubtitles f...,"[[-0.07956292, -0.014649572, 0.011612225, -0.0..."


In [ ]:
df3 = df1.loc[1001:5000,:]
df3.head()

,num,name,cleaned_text,chunks
1001,9185846,the.rising.s01.e03.episode.1.3.(2022).eng.1cd,i my daughter hasnt come home her names neve k...,[i my daughter hasnt come home her names neve ...
1002,9185847,the.rising.s01.e04.episode.1.4.(2022).eng.1cd,i they say you were murdered theres so much i ...,[i they say you were murdered theres so much i...
1003,9185848,the.rising.s01.e05.episode.1.5.(2022).eng.1cd,i all i can think about is neve whatevers happ...,[i all i can think about is neve whatevers hap...
1004,9185849,the.rising.s01.e06.episode.1.6.(2022).eng.1cd,i he cant even see you he can now i dont know ...,[i he cant even see you he can now i dont know...
1005,9185850,the.rising.s01.e07.episode.1.7.(2022).eng.1cd,i we found traces of blood in room seven of ke...,[i we found traces of blood in room seven of k...


In [ ]:
df3['embedding'] = df3['chunks'].apply(encode_text)
df3.head()

<ipython-input-85-b9e7c49d4141>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['embedding'] = df3['chunks'].apply(encode_text)


,num,name,cleaned_text,chunks,embedding
1001,9185846,the.rising.s01.e03.episode.1.3.(2022).eng.1cd,i my daughter hasnt come home her names neve k...,[i my daughter hasnt come home her names neve ...,"[[-0.009592683, -0.023190077, -0.029881032, -0..."
1002,9185847,the.rising.s01.e04.episode.1.4.(2022).eng.1cd,i they say you were murdered theres so much i ...,[i they say you were murdered theres so much i...,"[[-0.012840219, -0.032837816, 0.023085847, -0...."
1003,9185848,the.rising.s01.e05.episode.1.5.(2022).eng.1cd,i all i can think about is neve whatevers happ...,[i all i can think about is neve whatevers hap...,"[[0.0042446796, -0.07342178, -0.06385927, -0.0..."
1004,9185849,the.rising.s01.e06.episode.1.6.(2022).eng.1cd,i he cant even see you he can now i dont know ...,[i he cant even see you he can now i dont know...,"[[-0.058396105, -0.01499991, 0.0241822, -0.042..."
1005,9185850,the.rising.s01.e07.episode.1.7.(2022).eng.1cd,i we found traces of blood in room seven of ke...,[i we found traces of blood in room seven of k...,"[[-0.04492488, 0.020621229, -0.0023750784, -0...."


In [ ]:
df1.head()

,num,name,cleaned_text,chunks
0,9180533,the.message.(1976).eng.1cd,watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,ah theres princess dawn and terry with the blo...,[ah theres princess dawn and terry with the bl...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,yumis cells episode extremely polite yumi yumi...,[yumis cells episode extremely polite yumi yum...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...
4,9180600,broker.(2022).eng.1cd,i watch any video online with opensubtitles fr...,[i watch any video online with opensubtitles f...


In [ ]:
new_df = pd.concat([df2,df3],ignore_index=True, axis=0)
new_df.head()

,num,name,cleaned_text,chunks,embedding
0,9180533,the.message.(1976).eng.1cd,watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...,"[[-0.041853778, 0.12192455, -0.061531965, -0.0..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,ah theres princess dawn and terry with the blo...,[ah theres princess dawn and terry with the bl...,"[[-0.07573727, 0.012181397, 0.022164255, -0.07..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,yumis cells episode extremely polite yumi yumi...,[yumis cells episode extremely polite yumi yum...,"[[-0.13184361, -0.13974085, 0.045139585, -0.04..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...,"[[-0.08056183, -0.0905083, 0.058143258, -0.030..."
4,9180600,broker.(2022).eng.1cd,i watch any video online with opensubtitles fr...,[i watch any video online with opensubtitles f...,"[[-0.07956292, -0.014649572, 0.011612225, -0.0..."


In [ ]:
new_df.shape

In [ ]:
new_df.to_csv('df1.csv',index=False)

In [ ]:
df4 = df1.loc[5001:10000,:]


In [ ]:
df4['embedding'] = df4['chunks'].apply(encode_text)
df4.head()

<ipython-input-91-df27b8bcfb16>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['embedding'] = df4['chunks'].apply(encode_text)


,num,name,cleaned_text,chunks,embedding
5001,9203075,reign.s01.e22.slaughter.of.innocence.(2014).en...,i previously on reign have you seen him have y...,[i previously on reign have you seen him have ...,"[[-0.07053311, 0.012997416, 0.022657974, -0.00..."
5002,9203077,reign.s02.e01.the.plague.(2014).eng.1cd,i man previously on reign is that my husbands ...,[i man previously on reign is that my husbands...,"[[-0.046272963, 0.00058776594, -0.0070489524, ..."
5003,9203078,reign.s02.e02.drawn.and.quartered.(2014).eng.1cd,i francis previously on reign we have confirme...,[i francis previously on reign we have confirm...,"[[-0.090038046, 0.011583118, 0.016562684, -0.0..."
5004,9203079,reign.s02.e03.coronation.(2014).eng.1cd,i francis previously on reign catherine for go...,[i francis previously on reign catherine for g...,"[[-0.019157322, 0.006892328, 0.1009151, -0.059..."
5005,9203080,reign.s02.e04.the.lamb.and.the.slaughter.(2014...,i francis previously on reign give your baby y...,[i francis previously on reign give your baby ...,"[[0.019947879, -0.07666931, 0.008771424, -0.01..."


In [ ]:
new_df = pd.concat([new_df,df4],ignore_index=True, axis=0)
new_df.head()

,num,name,cleaned_text,chunks,embedding
0,9180533,the.message.(1976).eng.1cd,watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...,"[[-0.041853778, 0.12192455, -0.061531965, -0.0..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,ah theres princess dawn and terry with the blo...,[ah theres princess dawn and terry with the bl...,"[[-0.07573727, 0.012181397, 0.022164255, -0.07..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,yumis cells episode extremely polite yumi yumi...,[yumis cells episode extremely polite yumi yum...,"[[-0.13184361, -0.13974085, 0.045139585, -0.04..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...,"[[-0.08056183, -0.0905083, 0.058143258, -0.030..."
4,9180600,broker.(2022).eng.1cd,i watch any video online with opensubtitles fr...,[i watch any video online with opensubtitles f...,"[[-0.07956292, -0.014649572, 0.011612225, -0.0..."


In [ ]:
new_df.to_csv('df2.csv',index=False)

In [ ]:
df5 = df1.loc[10001:15000,:]

In [ ]:
df5['embedding'] = df5['chunks'].apply(encode_text)
df5.head()

<ipython-input-95-a14ae675ef46>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['embedding'] = df5['chunks'].apply(encode_text)


,num,name,cleaned_text,chunks,embedding
10001,9223793,recess.(1997).eng.1cd,bell rings door slammed and children screaming...,[bell rings door slammed and children screamin...,"[[-0.0014708587, -0.020333579, 0.079946116, -0..."
10002,9223794,recess.(1997).eng.1cd,bell rings children cheering waa umph support ...,[bell rings children cheering waa umph support...,"[[-0.060114015, -0.03532409, 0.001292536, -0.0..."
10003,9223795,recess.(1997).eng.1cd,bell ringing kids screaming uhoh advertise you...,[bell ringing kids screaming uhoh advertise yo...,"[[0.019813523, -0.08991281, 0.013590143, -0.00..."
10004,9223796,recess.(1997).eng.1cd,bell rings children cheer waah watch any video...,[bell rings children cheer waah watch any vide...,"[[-0.025653357, -0.09543994, 0.05376761, -0.07..."
10005,9223797,recess.(1997).eng.1cd,bell rings children cheer screams support us a...,[bell rings children cheer screams support us ...,"[[-0.07998118, -0.034499094, 0.011685411, -0.1..."


In [ ]:
new_df = pd.concat([new_df,df5],ignore_index=True, axis=0)
new_df.head()

,num,name,cleaned_text,chunks,embedding
0,9180533,the.message.(1976).eng.1cd,watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...,"[[-0.041853778, 0.12192455, -0.061531965, -0.0..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,ah theres princess dawn and terry with the blo...,[ah theres princess dawn and terry with the bl...,"[[-0.07573727, 0.012181397, 0.022164255, -0.07..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,yumis cells episode extremely polite yumi yumi...,[yumis cells episode extremely polite yumi yum...,"[[-0.13184361, -0.13974085, 0.045139585, -0.04..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...,"[[-0.08056183, -0.0905083, 0.058143258, -0.030..."
4,9180600,broker.(2022).eng.1cd,i watch any video online with opensubtitles fr...,[i watch any video online with opensubtitles f...,"[[-0.07956292, -0.014649572, 0.011612225, -0.0..."


In [ ]:
new_df.to_csv('embeddings.csv',index=False)

In [ ]:
df6 = df1.loc[15001:20000,:]

In [ ]:
new_df.to_csv('embeddings.csv',index=False)

In [ ]:
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_1 = pd.read_csv('/content/drive/My Drive/embeddings.csv')

In [ ]:
df_1.shape

(15001, 5)

In [ ]:
df_1.head()

,num,name,cleaned_text,chunks,embedding
0,9180533,the.message.(1976).eng.1cd,watch any video online with opensubtitles free...,['watch any video online with opensubtitles fr...,[[-0.04185378 0.12192455 -0.06153196 ... -0.0...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,ah theres princess dawn and terry with the blo...,['ah theres princess dawn and terry with the b...,[[-7.57372677e-02 1.21813966e-02 2.21642554e...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,yumis cells episode extremely polite yumi yumi...,['yumis cells episode extremely polite yumi yu...,[[-0.13184361 -0.13974085 0.04513958 ... 0.0...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,watch any video online with opensubtitles free...,['watch any video online with opensubtitles fr...,[[-0.08056183 -0.0905083 0.05814326 ... -0.0...
4,9180600,broker.(2022).eng.1cd,i watch any video online with opensubtitles fr...,['i watch any video online with opensubtitles ...,[[-0.07956292 -0.01464957 0.01161223 ... 0.1...


In [ ]:
! pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2

In [ ]:
import pandas as pd
import chromadb
import sqlite3
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
conn = sqlite3.connect('chromadb')

In [ ]:
import re

def preprocess(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove special characters
    text = re.sub(r'[^A-Za-z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra spaces and line breaks
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
df_1.to_sql('df_1',conn,if_exists='replace',index= False)

15001

In [ ]:
import sqlite3
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Connect to the SQLite database


# Load the DataFrame from the SQLite database
df_1 = pd.read_sql('SELECT * FROM df_1', conn)

# Getting user input
user_input = input("Enter your search query: ")

# Preprocess the user input
user_query = preprocess(user_input)

# Perform embedding on user query
model = SentenceTransformer('all-MiniLM-L6-v2')  # Adjust the BERT model name
user_embedding = model.encode([user_query])

# Perform embedding on DataFrame df_1
df_1_texts = df_1['chunks'].tolist()  # Adjust 'text_column' to the column containing text data
df_1_embeddings = model.encode(df_1_texts)

# Calculate cosine similarity between user embedding and df_1 embeddings
cos_similarities = cosine_similarity(user_embedding, df_1_embeddings)

# Add cosine similarities to df_1
df_1['cos_similarity'] = cos_similarities[0]

# Sort DataFrame by cosine similarity in descending order
df_1_sorted = df_1.sort_values(by='cos_similarity', ascending=False)

# Display the top similar results
top_results = df_1_sorted.loc[:, ['num','name','chunks']].head(10)  # Adjust the number of top results as needed
print(top_results)


Enter your search query: action


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

           num                                               name  \
5891   9206752  its.always.sunny.in.philadelphia.s09.e09.the.g...   
8117   9215530  my.huckleberry.friends.s01.e09.episode.1.9.()....   
1127   9186261    silver.spoon.s04.e04.episode.4.4.(2022).eng.1cd   
5301   9204135                             stachka.(1925).eng.1cd   
11845  9230801  ejen.ali.s01.e07.misi.protokol.gegas.(2016).en...   
14743  9242567  bleach.s06.e21.the.invisible.enemy.hitsugayas....   
3205   9195468       line.of.duty.s03.e01.monsters.(2016).eng.1cd   
7001   9210695  saturday.night.live.s01.e24.kris.kristofferson...   
13880  9239171          wet.lust.opening.the.tulip.(1975).eng.1cd   
14742  9242566  bleach.s06.e20.the.swooping.descent.of.the.dar...   

                                                  chunks  
5891   ['i aa aa support us and become vip member to ...  
8117   ['timing and subtitles brought to you by the t...  
1127   ['i watch any video online with opensubtitles ...  
5301

In [ ]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM df_1")
rows = cursor.fetchall()



In [ ]:
from sentence_transformers import SentenceTransformer, util


In [20]:
from chromadb.utils import embedding_functions
import chromadb
chroma_client = chromadb.PersistentClient(path="/content/drive/My Drive/my_chromadb1")
# Using the all-MiniLM-L6-v2 model for embedding function
embd = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection = chroma_client.get_or_create_collection(name="my_collection", embedding_function=embd, metadata={"hnsw:space": "cosine"})


In [21]:
collection.add(
    documents=df_1['name'].tolist(),
    metadatas=[{"item_id": str(idx)} for idx in range(len(df_1))],
    ids=[str(idx) for idx in range(len(df_1))],
)



In [18]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15001 entries, 0 to 15000
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   num           15001 non-null  int64 
 1   name          15001 non-null  object
 2   cleaned_text  15001 non-null  object
 3   chunks        15001 non-null  object
 4   embedding     15001 non-null  object
dtypes: int64(1), object(4)
memory usage: 586.1+ KB


In [19]:
import re

def preprocess(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove special characters
    text = re.sub(r'[^A-Za-z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra spaces and line breaks
    text = re.sub(r'\s+', ' ', text).strip()
    return text






In [ ]:
# getting user input
user_text = input("Enter your search query: ")

user_query = preprocess(user_text)
# querying the collection
results = collection.query(
    query_texts=[user_query],
    n_results=10,
    include=['documents', 'distances', 'metadatas']
)

# displaying the user input
print(f"Your search query: {user_query}")

# displaying output documents
for document, metadata in zip(results['documents'][0], results['metadatas'][0]):
    print(f"Document: {document}")
    print("Item id:", metadata['item_id'])

Enter your search query: love
Your search query: love


In [ ]:
results

{'ids': [[]],
 'distances': [[]],
 'metadatas': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'data': None}

In [ ]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import chromadb
from chromadb.utils import embedding_functions
import re



# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="my_chromadb")

# DistilBERT model for embedding function
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

# Get or create the collection
collection = chroma_client.get_or_create_collection(name="my_collection", embedding_function=sentence_transformer_ef, metadata={"hnsw:space": "cosine"})

# Set Streamlit page title and layout
st.set_page_config(page_title="Movie Verse", page_icon=":clapper:", layout="wide")

# Custom CSS for styling
st.markdown(
    """
    <style>
    body {
        background-color: #f0f2f6;
        color: #333333;
        font-family: Arial, sans-serif;
    }
    .stApp {
        max-width: 1000px;
        margin: auto;
        padding: 20px;
    }
    .stTextInput > div > div > div > input {
        border-radius: 20px;
        border: 2px solid #3498db;
        padding: 10px;
        outline: none;
    }
    .css-hi6a2p-singleValue {
        color: #3498db;
    }
    .css-1rhbuit-multiValue {
        background-color: #3498db;
    }
    .css-1rhbuit-multiValue__label {
        color: #ffffff;
    }
    .css-1uccc91-singleValue {
        color: #ffffff;
    }
    .stButton>button {
        border-radius: 20px;
        background-color: #3498db;
        color: #ffffff;
        font-weight: bold;
        padding: 10px 20px;
    }
    .stButton>button:hover {
        background-color: #2980b9;
    }
    </style>
    """,
    unsafe_allow_html=True
)

def main():
    st.title("🎬 Movie Verse 🎥")

    # Getting the user input
    user_query = st.text_input("Hey buddy, type your query and I will search movies for you:")

    if st.button("Search"):
        if user_query:
            # Preprocess the user query
            user_query = preprocess(user_query)

            # Query the collection
            results = collection.query(
                query_texts=[user_query],
                n_results=10,
                include=['documents', 'distances', 'metadatas']
            )

            # Display user input
            st.write(f"Your search query: {user_query}")

            # Display output documents
            st.write("Search Results:")
            for i, document in enumerate(results['documents'][0], 1):
                st.write(f"{i}. {document}")

if __name__ == "__main__":
    main()


In [ ]:
import streamlit as st
import chromadb
from chromadb.utils import embedding_functions

# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="my_chromadb")

# DistilBERT model for embedding function
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

# Get or create the collection
collection = chroma_client.get_or_create_collection(name="my_collection", embedding_function=sentence_transformer_ef, metadata={"hnsw:space": "cosine"})

# Set Streamlit page title and layout
st.set_page_config(page_title="Movie Verse", page_icon=":clapper:", layout="wide")

# Custom CSS for styling
st.markdown(
    """
    <style>
    body {
        background-color: #f0f2f6;
        color: #333333;
        font-family: Arial, sans-serif;
    }
    .stApp {
        max-width: 1000px;
        margin: auto;
        padding: 20px;
    }
    .stTextInput > div > div > div > input {
        border-radius: 20px;
        border: 2px solid #3498db;
        padding: 10px;
        outline: none;
    }
    .css-hi6a2p-singleValue {
        color: #3498db;
    }
    .css-1rhbuit-multiValue {
        background-color: #3498db;
    }
    .css-1rhbuit-multiValue__label {
        color: #ffffff;
    }
    .css-1uccc91-singleValue {
        color: #ffffff;
    }
    .stButton>button {
        border-radius: 20px;
        background-color: #3498db;
        color: #ffffff;
        font-weight: bold;
        padding: 10px 20px;
    }
    .stButton>button:hover {
        background-color: #2980b9;
    }
    </style>
    """,
    unsafe_allow_html=True
)

def main():
    st.title("🎬 Movie Verse 🎥")

    # Getting the user input
    user_query = st.text_input("Hey buddy, type your query and I will search movies for you:")

    if st.button("Search"):
        if user_query:
            # Query the collection
            results = collection.query(
                query_texts=[user_query],
                n_results=10,
                include=['documents', 'distances', 'metadatas']
            )

            # Display user input
            st.write(f"Your search query: {user_query}")

            # Display output documents
            st.write("Search Results:")
            for i, document in enumerate(results['documents'][0], 1):
                st.write(f"{i}. {document}")

if __name__ == "__main__":
    main()


2024-04-24 11:48:19.190 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
